For this project, I implemented stochastic gradient and adagrad on the multinomial logistic regression with squared 2-norm regularization and calculated the accuracy.





# Dataset

In [78]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random

In [79]:
# Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train_full, y_train_full), (x_test, y_test) = mnist.load_data()

# Normalize the data: scale the pixel values from 0-255 to 0-1
x_train_full, x_test = x_train_full / 255.0, x_test / 255.0

# Split the full training set into a smaller training set and a validation set
# For example, use 50,000 images for training and 10,000 for validation
x_train, x_valid = x_train_full[:50000], x_train_full[50000:]
y_train, y_valid = y_train_full[:50000], y_train_full[50000:]

# Now, x_train and y_train are the training set,
# x_valid and y_valid are the validation set,
# and x_test and y_test are the test set.

In [80]:
d = np.shape(x_train[1])        # The dimension of images
n = 10                          # Number of digits
N = len(x_train)                # Number of images in the traning dataset
Nv = len(x_valid)               # Number of images in the validation set
K = 30000                       # Number of iterations for the algorithm

In [81]:
# Convert labels to one-hot encoding
y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_valid_one_hot = tf.keras.utils.to_categorical(y_valid, num_classes=10)

# Calculating the gradient

$\forall i \in \{1, \dots, n\}$ The gradient of $F_i$ with respect to $w_{t,p} ~ \forall t \in \{1, \dots, d\} ~ \& ~ \forall p \in \{0, \dots, 9\}$ is $$\nabla_{w_{t, p}} F_i = x_{i,t}\frac{\exp~(\sum_{k = 1}^{d} x_{i,k}w_{k,p} + w_{0,p})}{\sum_{j = 0}^{9} \exp~(\sum_{k = 1}^{d} x_{i,k}w_{k,j} + w_{0,j})} - y_{i,p}x_{i,t} + w_{t, p}$$ The gradient of $F_i$ respect to $w_{0,p} ~ \forall p \in \{0, \dots, 9\}$ is $$\nabla_{w_{0,p}} F_i = \frac{\exp~(\sum_{k = 1}^{d} x_{i,k}w_{k,p} + w_{0,p})}{\sum_{j = 0}^{9} \exp~(\sum_{k = 1}^{d} x_{i,k}w_{k,j} + w_{0,j})} - y_{i,p}$$


In [82]:
def gradient(m, W, W0, alpha, X, Y):
    # m is the index of the image in X
    # W is the weight matrix of dimensions (d, n)
    # W0 is the bias vector of size n
    # alpha is the regularization parameter
    # X is the input data matrix
    # Y is the label matrix in one-hot encoding format

    G_W = np.zeros_like(W)  # Gradient with respect to W
    G_W0 = np.zeros_like(W0)  # Gradient with respect to W0

    # Compute gradients
    A = np.zeros(n)
    B = 0
    for j in range(n):
        A[j] = np.exp(np.vdot(X[m], W[:, :, j]) + W0[j])
        B += A[j]
    for j in range(n):
        G_W[:, :, j] = X[m] * (A[j] / B) - Y[m, j] * X[m] + alpha * W[:, :, j]
        G_W0[j] = A[j] / B - Y[m, j]

    return G_W, G_W0

# Stochastic Gradient

In [83]:
def stochastic_Gradient(alpha, W, W0, gamma, K, f, X, Y):
    for k in range(K):
        m = random.randint(0, N-1)         # Random index
        G = f(m, W, W0, alpha, X, Y)       # The gradient with respect to both W and W0
        W = W - gamma*G[0]                 # The W Update
        W0 = W0 - gamma*G[1]               # The W0 Update
    return W

In [97]:
W = stochastic_Gradient(0.7, np.zeros((d[0], d[1], n)), np.zeros(n), 1/np.sqrt(K), K, gradient, x_train, y_train_one_hot)

In [98]:
def Accuracy(W):
    Nt = N - Nv
    S = 0
    for i in range(Nt, N):
        argmax = np.argmax([(np.vdot(x_valid[i - Nt], W[:,:,j])) for j in range(n)]) # Find the index with the highest dot product
        if argmax == y_valid[i - Nt]:  # Check if the predicted label matches the actual label
            S += 1
    return S / Nv  # Calculate and return the accuracy

In [99]:
Accuracy(W)

0.6444

In [100]:
def find_best_alpha(alphas):
    best_alpha = None
    highest_accuracy = 0
    for alpha in alphas:
        W = stochastic_Gradient(alpha, np.zeros((d[0], d[1], n)), np.zeros(n), 1e-3/np.sqrt(K), K, gradient, x_train, y_train_one_hot)
        # Evaluate accuracy with the trained model
        current_accuracy = Accuracy(W)
        # Update best alpha if the current model performs better
        if current_accuracy > highest_accuracy:
            best_alpha = alpha
            highest_accuracy = current_accuracy
    return best_alpha, highest_accuracy

In [101]:
find_best_alpha(np.linspace(0.001, 3, 10))

(0.33422222222222225, 0.7478)

# Adagrad

In [89]:
def adagrad(f, W, W0, alpha, K, X, Y, epsilon=1e-8):
    # Initialize gradient accumulation vectors for W and W0
    G_W = np.zeros(W.shape)
    G_W0 = 0

    for k in range(K):
        m = random.randint(0, N-1)    # Random index
        G = f(m, W, W0, alpha, X, Y)  # The gradient with respect to both W and W0

        # Accumulate squared gradients
        G_W += G[0]**2
        G_W0 += G[1]**2

        # Update W and W0 with adaptive learning rates
        W = W - (alpha / (np.sqrt(G_W) + epsilon)) * G[0]
        W0 = W0 - (alpha / (np.sqrt(G_W0) + epsilon)) * G[1]

    return W

In [91]:
W = adagrad(gradient, np.zeros((d[0], d[1], n)), np.zeros(n), 0.7, K, x_train, y_train_one_hot)

In [92]:
Accuracy(W)

0.5491

In [93]:
def find_best_alpha_ada(alphas):
    best_alpha = None
    highest_accuracy = 0
    for alpha in alphas:
        W = adagrad(gradient, np.zeros((d[0], d[1], n)), np.zeros(n), alpha, K, x_train, y_train_one_hot)
        # Evaluate accuracy with the trained model
        current_accuracy = Accuracy(W)
        # Update best alpha if the current model performs better
        if current_accuracy > highest_accuracy:
            best_alpha = alpha
            highest_accuracy = current_accuracy
    return best_alpha, highest_accuracy

In [94]:
find_best_alpha_ada(np.linspace(0.001, 3, 10))

(0.001, 0.8501)